In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "test_ep2"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, return_dict=True, load_in_8bit=False, device_map="auto",torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/mpi/lib64'), PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/lib:/usr/local/mpi/lib:/usr/local/mpi/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('America/Los_Angeles')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/workspace/xglm_ok/Unti

In [3]:
batch = tokenizer('<human>: Evaluate the following statement " The world is better now compared to 10 years ago." <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1384: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 <human>: Evaluate the following statement " The world is better now compared to 10 years ago." <bot>: Absolutely. The world has become a much more peaceful place, with fewer wars and conflicts. The economy has improved, with more people able to afford basic necessities. The environment has been improved, with more people taking care of their natural resources. The quality of life has improved, with more people able to enjoy the fruits of their labor. Overall, the world has become a better place. The statement " The world is better now compared to 10 years ago." is a valid statement. It is a positive statement that reflects the positive changes that have occurred in the world over the past decade. It is a


In [4]:
batch

{'input_ids': tensor([[     2,   1131,  54987,   1128,     13, 140821,     90,     32,  12894,
          39730,     57,    268,   3038,     67,   4799,   1920,  67459,     33,
            209,   2432,   1939,   1406,   1131,   9932,   1128,     13,      6]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])}

In [5]:
batch = tokenizer('<human>: แนะนำวิธีลดน้ำหนักให้หน่อย <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: แนะนําวิธีลดน้ําหนักให้หน่อย <bot>: ลดน้ําหนักอย่างถูกต้อง involves a combination of healthy eating and exercise. Eating a healthy diet and exercising regularly will help you to lose weight and keep it off. You can also try to incorporate more physical activity into your daily routine, such as walking, jogging, or swimming. Additionally, you can try to cut down on your calorie intake and increase your physical activity. Finally, you can also try to reduce stress and anxiety, which can have a negative impact on your weight. Additionally, you can also try to take a look at your lifestyle and make changes if necessary. For example, if you are eating too


In [6]:
batch = tokenizer('Use Thai language only\n<human>: แนะนำวิธีลดน้ำหนักให้หน่อย <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Use Thai language only <human>: แนะนําวิธีลดน้ําหนักให้หน่อย <bot>: ลดน้ําหนักอย่างถูกวิธี ทําได้โดยปรับเปลี่ยนพฤติกรรมการกิน ออกกําลังกายอย่างสม่ําเสมอ ควบคุมอาหาร ดื่มน้ําให้เพียงพอ หลีกเลี่ยงอาหารที่มีไขมันสูง หลีกเลี่ยงอาหารที่มีน้ําตาลสูง หลีกเลี่ยงอาหารที่มีรสชาติหวานจัด หลีกเลี่ยงอาหารที่มีรสเค็มจัด หลีกเลี่ยงอาหารที่มีรสเผ็ดจัด หลีกเลี่ยงอาหารที่มีรสเปรี้ยวจัด หลีกเลี่ยงอาหารที่มีรสหวานจัด หลีกเลี่ยงอาหารที่มีรสเค็มจัด หลีกเลี่ยงอาหารที่มีรสเผ็ดจัด หลีกเลี่ยงอาหารที่มีรสเปรี้ยวจัด หลีกเล


In [7]:
batch = tokenizer('Use Thai language only without english\n<human>: อยากทำวิจัย ทำหัวข้ออะไรดี <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)


print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Use Thai language only without english <human>: อยากทําวิจัย ทําหัวข้ออะไรดี <bot>: การทําวิจัยในหัวข้อ "Thai language and culture" would be a great choice. It would allow you to explore the richness of Thai language and culture, as well as the uniqueness of Thai people. It would also provide you with a unique opportunity to learn about the Thai language and culture from a different perspective. Additionally, you could gain valuable insights into the Thai language and culture from the perspective of a foreigner. Finally, you could gain a deeper understanding of the Thai language and culture by conducting research on the topic. This would be a great opportunity to learn about the Thai language and culture from a different perspective


In [8]:
batch = tokenizer('<human>: อยากทำวิจัย ทำหัวข้ออะไรดี <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: อยากทําวิจัย ทําหัวข้ออะไรดี <bot>: การทํา research หัวข้ออะไรดี? ขึ้นอยู่กับวัตถุประสงค์ของการทํา research แต่ละคน เช่น การทํา research เพื่อหาข้อมูลเพื่อทํารายงาน การทํา research เพื่อหาความรู้เพิ่มเติม การทํา research เพื่อหาไอเดียใหม่ๆ ในการทําธุรกิจ เป็นต้น.. การทํา research แต่ละหัวข้อก็จะมีขั้นตอนและกระบวนการที่แตกต่างกันออกไป ขึ้นอยู่กับวัตถุประสงค์ของการทํา research แต่ละคน.. สุดท้ายนี้ การทํา research ถือเป็นทักษะที่สําคัญที่จะช่วยให้เราสามารถหาข้อมูลและความรู้ใหม่ๆ มาใช้ได้อย่างมีประสิทธิภาพ.. ขอให้สนุกกับการทํา research นะคะ.. وششوششوششوششوششوششوششوششوششوششوششوششوششوشش


In [9]:
batch = tokenizer('Use Thai language only\n<human>: ขอวิธีจีบผู้หญิงให้หน่อย <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Use Thai language only <human>: ขอวิธีจีบผู้หญิงให้หน่อย <bot>: อันดับแรก, you need to be confident in yourself. You need to be confident in your abilities and your personality. You need to be confident in your looks and your style. You need to be confident in your body language and your tone of voice. You need to be confident in your ability to make a good impression. You need to be confident in your ability to communicate effectively. You need to be confident in your ability to make a good first impression. You need to be confident in your ability to be yourself. You need to be confident in your ability to be assertive. You need to be confident in your


In [10]:
batch = tokenizer('<human>: ขอวิธีจีบผู้หญิงให้หน่อย <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: ขอวิธีจีบผู้หญิงให้หน่อย <bot>: เทคนิคในการจีบผู้หญิงให้สําเร็จมีดังนี้: 1. สร้างความประทับใจตั้งแต่แรกพบ ด้วยการแต่งตัวให้ดูดี แต่งหน้าให้ดูสวย แต่งตัวให้เข้ากับสถานที่ พูดจาสุภาพ อ่อนน้อมถ่อมตน 2. สร้างความมั่นใจให้กับตัวเอง ด้วยการพูดคุยกับเธออย่างเป็นธรรมชาติ กล้าแสดงออก กล้าถามคําถาม กล้าแสดงความคิดเห็น 3. สร้างความประทับใจด้วยคําพูด ด้วยการพูดจาไพเราะ พูดจาให้เธอรู้สึกดี พูดจาให้เธอรู้สึกประทับใจ 4. สร้างความประทับใจด้วยการกระทํา ด้วยการทําให้เธอรู้สึกดี ด้วยการเอาใจใส่เธอ ดูแลเธอ
